In [31]:
import ee
import geemap
from google.oauth2 import service_account

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import download_fctns

import masks
from masks import mask_MODIS_clouds, MODIS_Mask_QC, mask_s2_clouds, mask_s2_clouds_collection, worldcereal_mask


# Path to the private key file
key_path = 'Access/ee-martinparker637-e68fde65abb4.json'

# Load the service account credentials
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=['https://www.googleapis.com/auth/earthengine'])

# Initialize Earth Engine with the service account credentials
ee.Initialize(credentials)

In [32]:
import ee
import ee.batch
import eemont
import masks
import os
import pandas as pd
import numpy as np

def download_ndvi(year, grid_cell_lon, grid_cell_lat, filepath, crop_type='ww', N=10, country = 'DE', crop = 'Winter wheat'):
    
    """given a grid of coordinates, download the ndvi data masked to crop type, 
    for a 'representative field' for each grid cell"""
    country_codes = {'DE': 93,
                    'KEN': 133}
    product_codes = {'M': 'product == "maize"',
                     'ww': 'product == "wintercereals"'}
    save_path = filepath #f'ndvi_download/{crop_type}_{year}_{grid_cell_lon:.4f}_{grid_cell_lat:.4f}_s2.csv'
    #swap for engcrop when on gee
    #uk_outline = ee.FeatureCollection('FAO/GAUL/2015/level0').filterMetadata('ADM0_CODE','equals',256)
    country_outline = ee.FeatureCollection('FAO/GAUL/2015/level0').filterMetadata('ADM0_CODE','equals',country_codes[country])
    
    # cropland = ee.FeatureCollection("users/spiruel/crop_labels/CEH_cropmap2021")
    # crop = cropland.filterMetadata('crop_code','equals',crop_type)
    world_cereals = ee.ImageCollection('ESA/WorldCereal/2021/MODELS/v100')
    # Get a global mosaic for all agro-ecological zone (AEZ) of winter wheat
    crop = world_cereals.filter(product_codes[crop_type]).mosaic().select('classification').gte(0)

    # proj = ee.Projection('EPSG:27700')

    #for each grid cell, find the representative field
    #buffer each point by 12km (/2) (the scale of the GCM data)
    grid_cell = ee.Geometry.Point(grid_cell_lat, grid_cell_lon).buffer(12000//2).bounds()

    #get sentinel 2 ndvi data
    s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    s2 = s2.spectralIndices(['NDVI'])

    s2_filtered = masks.csPlus_mask_collection(s2).filterDate(f'{year-1}-01-01', f'{year}-12-31').filterBounds(country_outline)

    print(f'downloading ndvi for {crop_type}_{year}_{grid_cell_lon:.2f}_{grid_cell_lat:.2f}')
    #get the geometry of the field
    # geom = ee.Feature(crop).geometry()
    region = crop.clip(grid_cell).geometry()
    
    vectors = crop.reduceToVectors(**{
        'geometry': region,
        'scale': 100,
        'maxPixels': 1e13,
        'bestEffort':True,
        'eightConnected': False,
        })#.map(lambda x: x.buffer(-20))

    random_points = ee.FeatureCollection.randomPoints(**{'region': vectors, 'points': N, 'seed': 42, 'maxError': 1})

    #get the data for the field by reducing the image collection to the field geometry
    #get NDVI and spectral bands from S2
    s2_bands = ['NDVI','cs','cs_cdf']
    ts_s2 = s2_filtered.select(s2_bands).getTimeSeriesByRegion(**{
      'reducer': ee.Reducer.mean(),
      'geometry': random_points,#.buffer(10),
      'scale': 10,
      'maxPixels': 1e9
    })

    return ts_s2
    

In [14]:
root_directory = ''

In [15]:
coords = np.loadtxt(root_directory + "Saved_files/station_coords.csv", delimiter=',')

In [19]:
coord = coords[0]
year = 2019
filepath = 'Sentinel/Kenya/SatData0'

In [34]:
ts_s2 = download_ndvi(year, coord[1], coord[0], filepath, crop_type='M', N=10, country = 'DE')

downloading ndvi for ww_2019_9.43_54.78


In [35]:
#download the data
#try:
data_s2 = geemap.ee_to_df(ts_s2)
#except Exception as e:
#    print(e)
#    print(f'failed to download Maize_{year}_{coord[1]:.2f}_{coord[1]:.2f}')
data_s2 = data_s2[data_s2['NDVI']!=-9999]
#save the data
data_s2.to_csv(save_path, index=False)
print(f'saved {save_path}')

Exception: FeatureCollection.randomPoints: RandomPoints: Region must not be empty.

In [2]:
import geemap
geemap.ee_initialize(credentials)

In [19]:
class timeseries_downloader:
    def __init__(self, coords):
        self.coords = coords
        
    def initiate_image_collection(self, instrument = "COPERNICUS/S2_SR_HARMONIZED", bands = ['B4', 'B8'], 
                                  start_date = '2000-01-01', end_date = '2022-12-31', 
                                  QC_function = mask_s2_clouds_collection, pixel_scale = 500):
        self.bands = bands
        self.dataset = ee.ImageCollection(instrument).filterDate(start_date, end_date)
        self.dataset = QC_function(self.dataset)
        self.dataset = self.dataset.select(*bands)
        self.pixel_scale = pixel_scale
        
    def read_at_coords(self, box_width = 0.002):
        for coord_index, coord in enumerate(self.coords):
            print(coord_index)
            location = box_around_point(coord, box_width)
            filtered_dataset = self.dataset.filterBounds(location)
            filtered_dataset = reduce_region_collection(filtered_dataset, location, reducer_code = 'median', pixel_scale = self.pixel_scale)
            print('filtered!')
            df = collection_properties_to_frame(filtered_dataset, coord, self.bands, reducer_code = 'median')
            if coord_index == 0:
                self.df_full = df
            else:
                self.df_full = pd.concat([self.df_full, df])

def box_around_point(coord, box_width):
    '''
    coord: coordinate in (lat, lon) (?)
    '''
    return ee.Geometry.BBox(coord[1] - box_width, coord[0] - box_width, coord[1] + box_width, coord[0] + box_width)

def get_mean(image, location, pixel_scale= 500):
    return image.set('mean', image.reduceRegion(ee.Reducer.mean(), location , pixel_scale))

def get_median(image, location, pixel_scale= 500):
    return image.set('median', image.reduceRegion(ee.Reducer.median(), location , pixel_scale))

def reduce_region_collection(image_collection, location, reducer_code = 'median', pixel_scale = 500):
    if reducer_code == 'mean':
        return image_collection.map(lambda img: get_mean(img, location, pixel_scale = pixel_scale))
    elif reducer_code == 'median':
        return image_collection.map(lambda img: get_median(img, location, pixel_scale = pixel_scale))
    else:
        print('invalid reducer code')

def collection_properties_to_frame(image_collection, coord, bands):
    print('collecting...')
    timelist = image_collection.aggregate_array('system:time_start').getInfo()
    print('Times retrieved!')
    bandlist = image_collection.aggregate_array('median').getInfo()
    print('Data retrieved!')
    data = {'Time': timelist,
            'lat': [coord[0] for count in range(len(timelist))],
            'lon': [coord[1] for count in range(len(timelist))],
            'Stations_Id': [np.int64(coord[2]) for count in range(len(timelist))]
           }
    for band in bands:
        data[f'{reducer_code} {band}'] = [band_data[band] for band_data in bandlist]
    df = pd.DataFrame(data)
    df['formatted_time'] = pd.to_datetime(df['Time'], unit='ms').dt.strftime('%Y-%m-%d-%H-%M-%S')
    return df

SyntaxError: invalid syntax (2999962041.py, line 18)

In [3]:
startpoint = 0
endpoint = 2
savename = 'Sentinel/Kenya/SatData0'
print(startpoint, endpoint, savename)

root_directory = ''#'home/users/wlwc1989/phenology_dwd/'

#coords = np.loadtxt(root_directory + "Saved_files/station_coords.csv", delimiter=',')
coords = []
for lat in np.arange(-1, 1, 0.05):
    for lon in np.arange(35.05, 37.05, 0.05):
        coords.append([lat , lon, 0])

Sentinel_downloader = download_fctns.timeseries_downloader(coords[startpoint:endpoint])
Sentinel_downloader.initiate_image_collection(
    instrument = "COPERNICUS/S2_SR_HARMONIZED", bands = ['B4', 'B8'],
    start_date = '2020-01-01', 
    QC_function = mask_s2_clouds_collection, pixel_scale = 250)
#    QC_function = lambda IC: IC.map(MODIS_Mask_QC).map(mask_MODIS_clouds))
Sentinel_downloader.read_at_coords(box_width = 0.01)
Sentinel_downloader.df_full.dropna().to_csv(root_directory + f"Saved_files/{savename}.csv")

#df.dropna().to_csv(root_directory + f"Saved_files/{savename}.csv")


0 2 Sentinel/Kenya/SatData0
0
1


In [ ]:
MODIS_downloader = download_fctns.timeseries_downloader(coords[startpoint:endpoint])
MODIS_downloader.initiate_image_collection(
    instrument = "MODIS/061/MOD09GA", bands = [f'sur_refl_b0{n}' for n in range(1, 5)],
    start_date = '2020-01-01', 
    QC_function = lambda IC: IC.map(MODIS_Mask_QC).map(mask_MODIS_clouds).map(worldcereal_mask))
#    QC_function = lambda IC: IC.map(MODIS_Mask_QC).map(mask_MODIS_clouds))
MODIS_downloader.read_at_coords(box_width = 0.002)

In [9]:
import os

In [11]:
os.getcwd()

'/home/users/wlwc1989/earth_engine_MP'

In [15]:
savename = 'Sentinel/Kenya/SatData0'
MODIS_downloader.df_full.dropna().to_csv(root_directory + f"Saved_files/{savename}.csv")

In [6]:
url = 'https://github.com/ucg-uv/research_products/blob/main/cropcalendars/M1_EOS_WGS84.tif'
filename = 'Useful Files/calendar.tif'
geemap.download_file(url, filename)

Useful Files/calendar.tif already exists. Skip downloading. Set overwrite=True to overwrite.


'/home/users/wlwc1989/earth_engine_MP/Useful Files/calendar.tif'

In [8]:
arr = geemap.image_to_numpy(filename)

In [21]:
arr[0, np.int32(286/2), np.int32(720/2 + 70)]

335.0

In [16]:
img = ee.Image.loadGeoTIFF(url)

In [ ]:
Map = geemap.Map()
Map.add_raster(filename, band=[4, 1, 2], layer_name="Color infrared")
Map

In [2]:
root_directory = ''#'home/users/wlwc1989/phenology_dwd/'
#root_directory = 'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\'

In [3]:
coords = np.loadtxt(root_directory + "Saved_files/station_coords.csv", delimiter=',')

In [4]:
#coords = np.loadtxt("C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\station_coords.csv", delimiter=',')

In [6]:
def satellite_data_at_coords(coords, start_date = '2000-01-01', end_date = '2022-12-31', instrument = "COPERNICUS/S2_SR_HARMONIZED", bands = ['B4', 'B8'], box_width = 0.002, pixel_scale = 500, QC_function = mask_s2_clouds):
    for coord_index, coord in enumerate(coords):
        print(coord_index)
        location = ee.Geometry.BBox(coord[1] - box_width, coord[0] - box_width, coord[1] + box_width, coord[0] + box_width) #ee.Geometry.Point(coord[:2])#
        dataset = ee.ImageCollection(instrument)
        dataset = QC_function(dataset)
        filtered_dataset = dataset.filterDate(start_date, end_date).filterBounds(location)
        time_series_data = filtered_dataset.select(*bands).map(lambda img: img.set('median', img.reduceRegion(ee.Reducer.median(), location , pixel_scale)))#.getInfo()#.getRegion(location, pixel_scale).getInfo()#.getRegion(location, 30).getInfo()#('B4')#.median()#.get('B4')#reduceColumns(ee.Reducer.median().setOutputs(['median']), [''])#.get('B4')
        timelist = time_series_data.aggregate_array('system:time_start').getInfo()
        print('times retrieved')
        bandlist = time_series_data.aggregate_array('median').getInfo()
        print('bands retrieved')
        dataset = {'Time': timelist,
                   'lat': [coord[0] for count in range(len(timelist))],
                   'lon': [coord[1] for count in range(len(timelist))],
                   'Stations_Id': [np.int64(coord[2]) for count in range(len(timelist))]
                   }
        for band in bands:
            dataset[f'Median {band}'] = [band_data[band] for band_data in bandlist]
        df = pd.DataFrame(dataset)
        df['formatted_time'] = pd.to_datetime(df['Time'], unit='ms').dt.strftime('%Y-%m-%d-%H-%M-%S')
        if coord_index == 0:
            df_full = df
        else:
            df_full = pd.concat([df_full, df])
    return df_full

In [8]:
df = satellite_data_at_coords(coords[:2], start_date='2020-10-01', instrument = "MODIS/061/MOD09A1", QC_function = lambda IC: IC, bands = [f'sur_refl_b0{n}' for n in range(1, 5)]) #"MODIS/061/MOD09GA"

0
times retrieved
bands retrieved
1
times retrieved
bands retrieved


In [ ]:
df = satellite_data_at_coords(coords[:2], start_date='2020-10-01', instrument = "MODIS/061/MOD09A1", QC_function = lambda IC: IC.map(MODIS_Mask_QC).map(mask_MODIS_clouds), bands = [f'sur_refl_b0{n}' for n in range(1, 5)]) #"MODIS/061/MOD09GA"

In [22]:
fig, ax = plt.subplots()
for station in df['Stations_Id'].unique():
    df_reduced = df.where(df['Stations_Id'] == station).dropna()
    plt.plot(df_reduced['formatted_time'], df_reduced['Median B4'])

SyntaxError: expected ':' (1440415578.py, line 2)

In [47]:
#location = ee.Geometry.Point([10.05, 54.3167]) #(lon, lat)
Instrument = "MODIS/061/MOD09GA"
#Instrument = "MODIS/061/MOD09GQ"
#Instrument = "COPERNICUS/S2_SR_HARMONIZED"
coord = [54.3167, 10.05, 7504.0]
box_width = 0.002

location = ee.Geometry.BBox(8, 48, 15, 58)#ee.Geometry.BBox(coord[1] - box_width, coord[0] - box_width, coord[1] + box_width, coord[0] + box_width)
# Define the satellite imagery dataset (e.g., Landsat 8)
dataset = ee.ImageCollection(Instrument)

# Filter the dataset by location and date range
filtered_dataset = dataset.filterBounds(location).filterDate('2000-01-01', '2022-12-31')

In [52]:
filtered_dataset = filtered_dataset.map(MODIS_Mask_QC).map(mask_MODIS_clouds)

In [53]:
# Define a function to calculate NDVI
def calculate_ndvi(image, band1 = 'B4', band2 = 'B8'):
    ndvi = image.normalizedDifference([band1, band2]).rename('NDVI')
    return image.addBands(ndvi)

def get_median(image):
    return image.set('median', image.reduceRegion(ee.Reducer.median(), location , pixel_scale))

# Map the NDVI calculation function over the filtered dataset
ndvi_dataset = filtered_dataset.map(lambda image: calculate_ndvi(image, band1 = 'sur_refl_b01', band2 = 'sur_refl_b02'))

In [54]:
location = ee.Geometry.BBox(coord[1] - box_width, coord[0] - box_width, coord[1] + box_width, coord[0] + box_width) #ee.Geometry.Point(coord[:2])#
ndvi_dataset = ndvi_dataset.filterBounds(location)

In [27]:
ndvi_dataset = ndvi_dataset.map(mask_MODIS_clouds)

In [ ]:
pixel_scale = 500
bands = [f'sur_refl_b0{n}' for n in range(1, 5)]
bands.append('NDVI')
time_series_data = ndvi_dataset.select(bands).map(get_median)#.getInfo()#.getRegion(location, pixel_scale).getInfo()#.getRegion(location, 30).getInfo()#('B4')#.median()#.get('B4')#reduceColumns(ee.Reducer.median().setOutputs(['median']), [''])#.get('B4')
timelist = time_series_data.aggregate_array('system:time_start').getInfo()
print('times sorted')
bandlist = time_series_data.aggregate_array('median').getInfo()
print('bands sorted')
#QC_check = time_series_data.aggregate_array('state_1km').getInfo()
#print('QC sorted')
dataset = {'Time': timelist,
            'lat': [coord[0] for count in range(len(timelist))],
            'lon': [coord[1] for count in range(len(timelist))],
            'Stations_Id': [np.int64(coord[2]) for count in range(len(timelist))]
            }

times sorted


In [62]:
[f'sur_refl_b0{n}' for n in range(1, 5)].append(['NDVI'])

In [ ]:
bands = [f'sur_refl_b0{n}' for n in range(1, 5)]
bands.append('NDVI')
for band in bands:
    dataset[f'Median {band}'] = [band_data[band] for band_data in bandlist]
df = pd.DataFrame(dataset)
df['formatted_time'] = pd.to_datetime(df['Time'], unit='ms').dt.strftime('%Y-%m-%d-%H-%M-%S')

In [33]:
time_series_data = ndvi_dataset.select('NDVI', 'state_1km')#.map(get_median)
QC_check = time_series_data.aggregate_array('state_1km').getInfo()

In [ ]:
time_series_data = ndvi_dataset.select('NDVI').getRegion(location, 30).getInfo() 

In [58]:
coords[:10, :2]

array([[ 9.4333, 54.7833],
       [10.15  , 54.4   ],
       [10.05  , 54.3167],
       [10.6667, 53.8833],
       [10.7   , 53.85  ],
       [ 9.9833, 54.0833],
       [ 9.7   , 54.4333],
       [ 9.9833, 54.6167],
       [ 9.9667, 54.4   ],
       [ 9.85  , 54.4   ]])

In [63]:
#points = ee.Geometry.MultiPoint(coords[:2, :2].tolist())
location = ee.Geometry.Point([10.05, 54.3167])
time_series_data = filtered_dataset.select('B4', 'B8').getRegion(location, 30).getInfo()   #location, 30).getInfo()#